# <center>Cutting stock problem
---

### Importando bibliotecas

In [1]:
import json
import numpy as np
import random
import sys
from time import process_time

### Carregando dados de entrada

In [2]:
#Carregando .Json
with open("./data/1.json", encoding='utf-8') as arquivo:
    data = json.load(arquivo)

In [3]:
#Recuperando caixa/container, sua dimensção, e qtd disponível 
data['Objects']

[{'Length': 52, 'Height': 53, 'Stock': 3, 'Cost': 2756}]

In [4]:
#Recuperando itens e suas informações
data['Items']

[{'Length': 19, 'Height': 21, 'Demand': 1, 'DemandMax': None, 'Value': 399},
 {'Length': 7, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 140},
 {'Length': 4, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 80},
 {'Length': 15, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 300},
 {'Length': 14, 'Height': 20, 'Demand': 1, 'DemandMax': None, 'Value': 280},
 {'Length': 14, 'Height': 19, 'Demand': 1, 'DemandMax': None, 'Value': 266},
 {'Length': 17, 'Height': 19, 'Demand': 1, 'DemandMax': None, 'Value': 323},
 {'Length': 10, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 170},
 {'Length': 13, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 221},
 {'Length': 5, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 85},
 {'Length': 16, 'Height': 17, 'Demand': 1, 'DemandMax': None, 'Value': 272},
 {'Length': 20, 'Height': 16, 'Demand': 1, 'DemandMax': None, 'Value': 320},
 {'Length': 5, 'Height': 16, 'Demand': 1, 'DemandMax': None, 'Value': 80},
 {'Len

### Formatando dados

In [5]:
# Classe para os Containers
class Container:
    def __init__(self, Length, Height, Cost):  
        self.length = Length  
        self.height = Height
        self.cost   = Cost
        self.area   = np.zeros((self.length, self.height))
        self.util   = 0
        self.itens  = []

# Classe para as áreas maximais
class Area:
     def __init__(self, Length, Height, Corte):  
        self.length  = Length  
        self.height  = Height
        self.area    = self.length * self.height
        self.corte   = Corte

# Classe para os itens
class Item:
     def __init__(self, Length, Height, Corte):  
        self.length  = Length  
        self.height  = Height
        self.corte   = Corte

# Informações dos Containers e itens de entrada
infoObjs = data['Objects']
infoItns = data['Items']

In [6]:
# Lista dos itens de entrada
Lista_Itens = []

for itens in infoItns:
    for x in range(itens['Demand']):
        Lista_Itens.append(itens)

# Conjunto de Container de entrada
S = []

for objeto in infoObjs:
    for x in range(objeto['Stock']):
        S.append(Container(objeto['Length'],objeto['Height'],objeto['Cost']))
    
# qtd de caixa disponíveis
stock = len(S)
    
# Valor alpha
alpha = 0.3

### Heurística

In [7]:
def Heuristica():
    Objeto             = S[0]
    E                  = []
    bestsAM            = []
    AM_inicial         = Area(Objeto.length, Objeto.height, Objeto.area[:])
    E.append(AM_inicial)
    contador = 1
    
    for item in Lista_Itens:        
        for AM in E:
            if(Acomoda(AM, item)):
                bestsAM.append(AM)
                
        if(len(bestsAM) == 0):
            Objeto = S[contador]
            AreaObj = Area(Objeto.length, Objeto.height, Objeto.area[:])
            bestsAM.append(AreaObj)
            Empacotar(item, bestsAM, Objeto, alpha)
            E = Atualizar(Objeto)
            bestsAM = []
            if(contador > stock):
                break
            contador += 1

        else:
            Empacotar(item, bestsAM, Objeto, alpha)
            E = Atualizar(Objeto)
            bestsAM = []
            
    
def Acomoda(area, item):
    if(area.length >= item['Length'] and area.height >= item['Height'] ):
        return True
    else:
        return False
    
    
def Best(atualAM, melhorAM, item):
    if(melhorAM is None):
        return atualAM
    
    else:
        if(atualAM.length == item['Length'] and atualAM.height == item['Height']):
            return atualAM
        elif(melhorAM.length == item['Length'] and melhorAM.height == item['Height']):
            return melhorAM
        elif(atualAM.area > melhorAM.area):
            return atualAM
        else:
            return melhorAM
    
    
def Empacotar(item, melhorAM, Objeto, alpha):
    aleatorio = random.randint(0, int((alpha * (len(melhorAM))) - 1))
    
    AM       = melhorAM[aleatorio]
    corte    = AM.corte[:item['Length'], :item['Height']]
    corte[:] = 1
    
    Objeto.itens.append(Item(item['Length'], item['Height'], corte))
    Objeto.util += item['Length'] * item['Height']
    

def Atualizar(Objeto):
    AMs          = []
    aux          = np.empty_like(Objeto.area)
    aux[:]       = Objeto.area
    Inicial      = [0, 0]
    Final        = [0, 0]
    indicador    = 2
    
    for x in range(Objeto.length):  
        for y in range(Objeto.height):
            if(aux[x][y] == 0):
                area         = []
                corte   = aux[x:Objeto.length, y:Objeto.height]
                limX    = corte.shape[0]
                Inicial = [x, y]
                cont    = 0
                achou   = 0
                
                for k in range(corte.shape[0]):
                    for z in range(corte.shape[1]):
                        if corte[k][z] == 1:
                            achou = 1
                            
                            if k == 0 and cont == 0:
                                limX  = z
                                Final = [Objeto.length, limX-corte.shape[1]]
                                area  = aux[Inicial[0]:Final[0], Inicial[1]:Final[1]]
                                cont  = 1
                                break
                                
                            elif limX == corte.shape[0]:
                                Final = [corte.shape[0]-k, Objeto.height]
                                area  = aux[Inicial[0]:Final[0], Inicial[1]:Final[1]]
                                break
                            
                            elif cont == 1 and z < limX:
                                Final = [corte.shape[0]-k, limX-corte.shape[1]]
                                area  = aux[Inicial[0]:Final[0], Inicial[1]:Final[1]]
                                break
                        else:
                            continue
                            
                if achou == 0:
                    Final = [Objeto.length, Objeto.height]
                    area  = aux[Inicial[0]:Final[0], Inicial[1]:Final[1]]                 
                    
                area[:] = indicador
                AreaMaximal = Area(area.shape[0], area.shape[1], Objeto.area[Inicial[0]:Final[0], Inicial[1]:Final[1]])
                AMs.append(AreaMaximal)
                indicador += 1
    return AMs


def imprimirConjunto():
    print('\nQtd de Container =', len(S))
    cont = 1
    for i in S:
        print('\n\n\t\t', cont, '→', 'Length:', i.length, ' Height:', i.height, ' Área:', i.cost, ' Qtd de Itens:', 
              len(i.itens), ' Área Usada:', i.util,'\n\n')
        cont += 1
        np.set_printoptions(linewidth=150)
        np.set_printoptions(threshold=sys.maxsize)
        print(i.area.astype(int), '\n')

In [8]:
def Busca_Local():
    size = len(S) - 1
    E    = []
    
    for x in range(size):
        First  = None
        Second = None
        
        if(S[x].util > S[x+1].util):
            First   = S[x]
            Second  = S[x+1]
        else:
            First   = S[x+1]
            Second  = S[x]
        
        if(len(Second.itens) > 0):
            aleatorio   = random.randint(0, len(Second.itens) - 1)
            lista       = Second.itens[aleatorio:].copy()
        else:
            continue
            
        E = Atualizar(First)
        for item in lista:
            melhorAM = None
            for AM in E:
                if(AM.length >= item.length and AM.height >= item.height):
                    if(melhorAM is None):
                        melhorAM = AM
                    elif(AM.length == item.length and AM.height == item.height):
                        melhorAM = AM
                    elif(AM.area > melhorAM.area):
                        melhorAM = AM
                    
            if(melhorAM is None):
                continue
                
            else:
                addItem    = melhorAM.corte[:item.length, :item.height]
                addItem[:] = 1
                First.itens.append(Item(item.length, item.height, addItem))
                First.util += item.length * item.height
                
                item.corte[:] = 0
                Second.util -= item.length * item.height
                Second.itens.remove(item)
                E = Atualizar(First)

A vizinhança consiste em escolher duas caixas adjacentes do conjunto S. Será selecionado um ponto da lista de empacotamento da caixa que tiver menor área utilizada, e em seguida serão removidos todos os itens desta posição até o fim da lista. A reconstrução será a partir da solução parcial, notada na literatura como mais vantajosa. Tentaremos encaixar esses itens na caixa que tem maior área utilizada, a fim de maximar a maior qtd de itens possíveis, minimizando, consequentemente, os espaços livres, deixando a caixa mais "cheia". As decisões para selecionar as AM's que sobrou nas caixas para encaixar os itens utilizarão critérios gulosos. A busca local termina quando não houver mais vizinhança a serem comparadas.

In [9]:
start = process_time() 
Heuristica()
Busca_Local()
stop = process_time()
print("CPU-Time:", stop-start) 

CPU-Time: 0.65625


In [10]:
imprimirConjunto()


Qtd de Container = 3


		 1 → Length: 52  Height: 53  Área: 2756  Qtd de Itens: 11  Área Usada: 2276 


[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1 1 1 1